## DATA PREPARATION: IDENTIFICATION OF MCI SUBJECTS AND SCREENING OF TABLES

In [1]:
########## IMPORT LIBRARIES

import numpy as np
import pandas as pd
import os
import sys
import re

In [2]:
os.getcwd()

'/home/jovyan/work/DATA_PREPARATION/R_SCREENING_MASSI'

In [7]:
########## Import file for table selection 

# from the origianl file, I added a column "inclusion" with a x 
# for the table to be included, according to my manual screening
# DATADIC = pd.read_excel(io = "/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/temp_r/DATADIC.xlsx")
DATADIC = pd.read_excel(io = "/home/jovyan/work/ADNI_data/DATA_15_12_2017/temp_R/DATADIC.xlsx")
DATADIC = DATADIC.loc[DATADIC["inclusion"] == "x"]
DATADIC = DATADIC.loc[DATADIC["Phase"] != "ADNI3"]

In [8]:
########## Import ADNIMERGE FILE for baseline diagnosis and convertion at 3 years

# ADNIMERGE = pd.read_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/Study_Info/ADNIMERGE.csv")
ADNIMERGE = pd.read_csv("/home/jovyan/work/ADNI_data/DATA_15_12_2017/Study_Info/ADNIMERGE.csv")

# INCLUDE ONLY MCI
to_include_mci = np.unique(np.array(ADNIMERGE["RID"].loc[ADNIMERGE["DX_bl"].isin(["EMCI","LMCI"])]))
ADNIMERGE = ADNIMERGE.loc[ADNIMERGE["RID"].isin(to_include_mci)]

# INCLUDE ONLY SUBJECTS WITH INFO OF CONVERTION AT LEAST YEAR 3
to_include_time = ["m114","m126","m144","m102","m132","m120","m108","m90","m96","m84","m78","m66","m54","m42","m72","m60","m48","m36"]
to_include_subjects = np.unique(np.array(ADNIMERGE["RID"].loc[ADNIMERGE["VISCODE"].isin(to_include_time)]))
ADNIMERGE = ADNIMERGE.loc[ADNIMERGE["RID"].isin(to_include_subjects)]

# OF THESE SUBJECTS, KEEP ONLY INFO OF CONVERTION BY YEAR 3
to_include_time_2 = ["bl","m03","m30","m36","m24","m18","m12","m06"]
ADNIMERGE = ADNIMERGE.loc[ADNIMERGE["VISCODE"].isin(to_include_time_2)]

In [9]:
########## Import OTHER FILE with relevant data, select only relevant data

names_table_included = np.append(np.unique(np.array(DATADIC['TBLNAME'])),"CBB")

files_all = np.array([])
# for path, subdirs, files in os.walk("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017"):
for path, subdirs, files in os.walk("/home/jovyan/work/ADNI_data/DATA_15_12_2017"):
    for name in files:
        files_all = np.append(files_all,os.path.join(path, name))

# keep only csv file and those identified in names_table_included
files_all = pd.Series(files_all)
files_all = files_all[files_all.str.endswith(".csv")]
files_all = files_all[files_all.str.contains("|".join(names_table_included))]
files_all.index = np.array(range(files_all.shape[0]))

# for each csv file, create the DataFrame and include only baseline info of only MCI subjects
for i in range(files_all.shape[0]):
    name_table_this = re.sub(".*/","",re.sub(".csv","",files_all[i]))
    table_this = pd.read_csv(files_all[i])
    
    # include only variable at baseline or screening
    if "VISCODE" in table_this.columns:
        table_this = table_this.loc[table_this["VISCODE"].isin(["sc","bl","v03"])]
        
    # INCLUDE only baseline MCI subjects
    if "RID" in table_this.columns:
        table_this = table_this.loc[table_this["RID"].isin(to_include_subjects)]
        if table_this["RID"].size > np.unique(np.array(table_this["RID"])).size:
            print(name_table_this + " may be in long format and not in wide format")
            
    # save it as a dataframe only if it has at least more than 10 rows
    if table_this.shape[0] > 10:
         exec(name_table_this + " = table_this") 

# cleansing
del table_this
del name_table_this
del DATADIC

AV45VITALS may be in long format and not in wide format
VITALS may be in long format and not in wide format
RECMHIST may be in long format and not in wide format
BACKMEDS may be in long format and not in wide format


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (9,15,16,18,19,21,22,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


RECCMEDS may be in long format and not in wide format
UWNPSYCHSUM_10_27_17 may be in long format and not in wide format
NEUROBAT may be in long format and not in wide format


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


ADNI_CBBRESULTS may be in long format and not in wide format


/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,58,59,60,61,62,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


PTDEMOG may be in long format and not in wide format
RECFHQ may be in long format and not in wide format


In [10]:
########## DECIDE NOW WHAT TO DO FOR EACH TABLE: REMOVE SOME TABLES AND MERGE OTHERS

# RESHAPE ADNIMERGE AND ADD CONVERTION VARIABLE AT THE END
tempo = ADNIMERGE.copy()
tempo.index = np.array(range(tempo.shape[0]))
ADNIMERGE = ADNIMERGE.loc[ADNIMERGE['VISCODE'].isin((["sc","bl","v03"]))]
ADNIMERGE.index = np.array(range(ADNIMERGE.shape[0]))

ADNIMERGE["CONVERSION_AT_3Y"] = "NO"

for i in range(ADNIMERGE.shape[0]):
    if "Dementia" in np.array(tempo["DX"].loc[tempo["RID"] == ADNIMERGE["RID"].iloc[i]]):
        ADNIMERGE["CONVERSION_AT_3Y"].iloc[i] = "YES"
        
# remove ADAS: the total scores of the two ADAS battery can be found in the two ADASSCORES AND ADAS_ADNIGO23 tables (one for adni1 and the other for adnigo23) can already be found in ther ADNIMERGE table
# what is missing is the single item score we will include eventually later
del ADAS_ADNI1
del ADAS_ADNIGO23
del ADASSCORES

# remove ADNI_CBBRESULTS is a cognitive battery used only in ADNI2 (only around 50 subjects)
del ADNI_CBBRESULTS

# ADSXLIST and BLSCHECK are the same, with different names for different ADNI phase.
BLSCHECK.columns = ADSXLIST.columns
ADSXLIST = pd.concat([ADSXLIST, BLSCHECK])
del BLSCHECK

# remove AV45VITALS as it seems to be data related to a challange 
del AV45VITALS

# reshape BACKMEDS. it is the coding key medications taken by ADNIGO23 only subjects
BACKMEDS["MED_0"] = 0
BACKMEDS["MED_1"] = 0
BACKMEDS["MED_2"] = 0
BACKMEDS["MED_3"] = 0
BACKMEDS["MED_4"] = 0
BACKMEDS["MED_5"] = 0
BACKMEDS["MED_6"] = 0
BACKMEDS["MED_7"] = 0

BACKMEDS["MED_0"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("0")] = 1
BACKMEDS["MED_1"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("1")] = 1
BACKMEDS["MED_2"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("2")] = 1
BACKMEDS["MED_3"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("3")] = 1
BACKMEDS["MED_4"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("4")] = 1
BACKMEDS["MED_5"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("5")] = 1
BACKMEDS["MED_6"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("6")] = 1
BACKMEDS["MED_7"].loc[pd.Series(BACKMEDS["KEYMED"]).str.contains("7")] = 1

# remove BLCHANGE. It codes the change in other info than diagnosis
del BLCHANGE

# keep CDR. subscores have already proved to be predicted

# remove CLIELG. Inclusion criteria satisfaction

# remove ECOGPT. it is the item scores of everyday cog participant. Total is already in adnimerge
del ECOGPT

# remove ECOGSP. it is the item scores of everyday cog partner. Total is already in adnimerge
del ECOGSP

# remove FAQ. It contains item levels FAQ but total already in ADNIMERGE
del FAQ

# keep DXSUM_PDXSUM_ADNIALL. diagnostic summary and specifics

# keep FHQ. Family history questionnaire

# keep GDSCALE. Geriatric depression cale but seems not for all included participants

# keep MEDHIST. Medical history for ADNI1GO

# remove MMSE. it's the item level of MMSE and the total is included in the ADNIMERGE
del MMSE

# remove MOCA. it is the item scores of MOCA. Total is already in adnimerge
del MOCA

# keep MODHACH. Modified Hachinski test

# keep NEUROBAT. Several tests not included in ADNIMERGE

# keep NEUROEXM. Neurological examination at screening

# merge NPI and NPIQ in NPI. NEUROPSYCHIATRIC info for ADNI2 not included in ADNIMERGE
# keep only total scores
# "SEV" in var name means severity so I change names ending with TOT in SEV for consistency between the two tables
NPI.columns = [re.sub("TOT$","SEV",elem) for elem in NPI.columns]
NPI.columns = [re.sub("NPITOTAL","NPISCORE",elem) for elem in NPI.columns]
NPI.columns = [re.sub("NPISOURC$","NPISOURCE",elem) for elem in NPI.columns]

NPI = NPI[NPIQ.columns]

NPI = pd.concat([NPI, NPIQ])

del NPIQ

# keep PHYSICAL. Physical screening

# keep PTDEMOG. Sociodemographics info

# keep RECCMED. Information about medications for all included subjects, but very hard to recode and originally in long format

# keep RECFHQ. Information about familiarity for dementia for all included subjects, but to be recoded and in long format

# keep RECMHIST. Information about medical history for all included subjects, but very hard to recode and originally in long format

# keep UWNPSYCHSUM_10_27_17. These are new total neurospychology scores for memory and exec func

# keep VITALS


/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [12]:
########## FINAL CLEANSING
del i
del names_table_included


NameError: name 'i' is not defined

In [13]:
ADNIMERGE.shape

(550, 110)

In [20]:
########## EXPORT NEW TABLES

ADNIMERGE.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/ADNIMERGE.csv")
ADSXLIST.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/ADSXLIST.csv")
BACKMEDS.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/BACKMEDS.csv")
CDR.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/CDR.csv")
CLIELG.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/CLIELG.csv")
DXSUM_PDXCONV_ADNIALL.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/DXSUM_PDXCONV_ADNIALL.csv")
FHQ.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/FHQ.csv")
GDSCALE.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/GDSCALE.csv")
MEDHIST.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/MEDHIST.csv")
MODHACH.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/MODHACH.csv")
NEUROBAT.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/NEUROBAT.csv")
NEUROEXM.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/NEUROEXM.csv")
NPI.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/NPI.csv")
PHYSICAL.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/PHYSICAL.csv")
PTDEMOG.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/PTDEMOG.csv")
RECCMEDS.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/RECCMEDS.csv")
RECFHQ.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/RECFHQ.csv")
RECMHIST.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/RECMHIST.csv")
UWNPSYCHSUM_10_27_17.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/UWNPSYCHSUM_10_27_17.csv")
VITALS.to_csv("/Users/massimilianograssi/Google Drive/Joint_projects_DS_Massi/1_ADNI_MCI_to_AD/DATA_15_12_2017/SCREENED_TABLES/VITALS.csv")



